In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Conv1D, GlobalMaxPooling1D, Flatten, concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD

import torch
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import pickle

In [24]:
with open("D:\\LICENTA2024\\datasetsABSA\\SemEval-2016 Task 5\\Restaurants\\restaurants_data_2016", "rb") as fp:
    sentences_data = pickle.load(fp)

print(sentences_data[0])
print(sentences_data[1])
print(sentences_data[2])


('Judging from previous posts this used to be a good place, but not any longer.', 'RESTAURANT', 'negative')
('We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.', 'SERVICE', 'negative')
('They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.', 'SERVICE', 'negative')


In [25]:
clauses = [data[0] for data in sentences_data]
categories = [data[1] for data in sentences_data]
sentiments = [data[2] for data in sentences_data]

In [26]:
# Split the data into training and validation data
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(clauses, categories, sentiments, test_size=0.2, random_state=42)

In [27]:
from transformers import BartTokenizer, BartModel
import torch

def vectorize(clauses, file_name):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    model = BartModel.from_pretrained('facebook/bart-large')

    clause_vectors = []
    nr = 0

    for clause in clauses:
        inputs = tokenizer(clause, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)

        last_hidden_state = outputs.last_hidden_state

        clause_vectors.append(last_hidden_state)
        print("------------------------>", "Done sentence ", nr, "<----------------------------")
        nr += 1

    print(clause_vectors[:2])
    with open(file_name, "wb") as fp:   #Pickling
        pickle.dump(clause_vectors, fp)

In [28]:
vectorize(X_train, "X_train_bart")
vectorize(X_test, "X_test_bart")

------------------------> Done sentence  0 <----------------------------
------------------------> Done sentence  1 <----------------------------
------------------------> Done sentence  2 <----------------------------
------------------------> Done sentence  3 <----------------------------
------------------------> Done sentence  4 <----------------------------
------------------------> Done sentence  5 <----------------------------
------------------------> Done sentence  6 <----------------------------
------------------------> Done sentence  7 <----------------------------
------------------------> Done sentence  8 <----------------------------
------------------------> Done sentence  9 <----------------------------
------------------------> Done sentence  10 <----------------------------
------------------------> Done sentence  11 <----------------------------
------------------------> Done sentence  12 <----------------------------
------------------------> Done sentence  13 <---

In [29]:
with open("D:\\LICENTA2024\\datasetsABSA\\SemEval-2016 Task 5\\Restaurants\\X_train_bart", "rb") as fp:
    X_train_bart = pickle.load(fp)

with open("D:\\LICENTA2024\\datasetsABSA\\SemEval-2016 Task 5\\Restaurants\\X_test_bart", "rb") as fp:
    X_test_bart = pickle.load(fp)

In [32]:
model_raw_data = (X_train, X_train_bart,  X_test, X_test_bart, y1_train, y1_test, y2_train, y2_test)
with open("model_raw_data", "wb") as fp:   #Pickling
    pickle.dump(model_raw_data, fp)